<a href="https://colab.research.google.com/github/nrjcs/iitpbse/blob/master/dl/d2/bse_own_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Image classification using our own data with Keras



> Make sure that the data (with which you would like to perform classification) is available in your Google Drive

> Data used in this example, is a subset of Kaggle dataset (available @ https://www.microsoft.com/en-us/download/details.aspx?id=54765)

> You can download the dataset which is available at the above mentioned link. Alternatively, you can download the dataset from the link https://github.com/nrjcs/mlip/raw/master/d4/img_data_kaggle.zip

> In any case, the dataset (images and not the compressed file) must be uploaded in your google drive  in the directory "img_data"...if you would like to put the data in different directory then update ip_path variable in incoming cell 

> Data consists of 100 images of cats and dogs each





In [ ]:
#intialization 
train_labels=[]
train_samples=[]


**load the Drive helper and mount**

following code cell is to link your drive with Colab...the procedure for the same is as follows:

> execute this code cell -> click the link that you will get as output -> select the account to sign with which the Google Drive is associated -> copy the code -> (switch back to this page) -> paste the copied code in the box following the message "Enter your authorization code:" in the output pane on execution of following code you will get a link -> press enter

In [ ]:

from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/img_data/"

- Create a directory named "img_data_updated" to store the converted images

In [ ]:
!ls "/content/drive/My Drive/img_data_updated/" # the colored images are converted and stored in this directory

In [ ]:
# initialization of input and output paths of images
ip_path='/content/drive/My Drive/img_data/'
op_path='/content/drive/My Drive/img_data_updated/'

In [ ]:
import os # Miscellaneous operating system interfaces...file and directory related operations among others
from numpy import * #package for scientific computing

#getting list of images
listing=os.listdir(ip_path) #list of the names of the entries in the arg (directory)
num_samples=size(listing)
print(num_samples)


In [ ]:
#intiatialization
img_rows, img_cols=64, 64
channels=1


from PIL import Image # PIL is the Python Imaging Library ....adds image processing capabilities to your Python interpreter....visit https://pillow.readthedocs.io/en/stable/
#image preprocessing...resize...convert to grayscale....and save to output directory
for file in listing:
    im=Image.open(ip_path+'/'+file)
    img=im.resize((img_rows,img_cols))
    gray=img.convert('L')
    gray.save(op_path+'/'+file, "JPEG")
imlist=os.listdir(op_path)  
print (imlist)

In [ ]:
# sort....
imlist.sort(key=lambda f: int(filter(str.isdigit, f)))
print (imlist)

In [ ]:
# viewing images

import matplotlib.pyplot as plt

im1=array(Image.open('/content/drive/My Drive/img_data_updated/'+imlist[0]))
plt.imshow(im1)
#plt.imshow(im1,cmap='gray')
print(im1.shape)
m,n=im1.shape[0:2]
print m,m

In [ ]:
#flatten all the images into one matrix
img_mat=array([array(Image.open('/content/drive/My Drive/img_data_updated/'+imseq)).flatten() 
           for imseq in imlist],'f')

#print img_mat
#print img_mat.shape

In [ ]:
import numpy as np
# initialize labels
labels=np.ones((num_samples),dtype=int)
print labels

In [ ]:
#reset labels
labels[0:100]=0
labels[100:200]=1
print labels

In [ ]:
from sklearn.utils import shuffle
#make sample label pairs
data,label=shuffle(img_mat,labels)
#for i in range(200):
#   print data[i],label[i]

In [ ]:
#combine data and labels as single input

#this step in not necessary as we can input the data and label as two arrarys
train_data=[data,label]
print train_data[0].shape #shape of samples

print train_data[1].shape #shape of labels
print train_data



In [ ]:
#check the images from the flattened matrix
check_img=img_mat[11].reshape(img_rows,img_cols)
plt.imshow(check_img)
plt.imshow(check_img,cmap='gray')

In [ ]:
(X,Y)=(train_data[0],train_data[1])  

from sklearn.model_selection import train_test_split
# preprocessing
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state=4)

X_temp = X_test

print (X_train.shape)

# flatten 28*28 images to a 784 vector for each image and pixel precision set to 32 bit
num_pixels = img_rows * img_cols
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

from keras.utils import np_utils		#for transforming data 

# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)
num_classes = Y_test.shape[1]

# check again	
print (Y_train.shape)
# (60000, 10)
print (Y_train[:5])

In [ ]:
input_dim = img_rows * img_cols
output_dim = 2
# Define model architecture

from keras.models import Sequential		#model
from keras.layers import Dense			#layer
from keras.layers import Dropout		#layer
from keras import initializers      # for importing initializers of keras

model = Sequential()
model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer=initializers.RandomNormal(), activation='relu')) #only one hidden layer with relu as activation function
#model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer=initializers.Constant(value=5), activation='relu')) #only one hidden layer with relu as activation function	
model.add(Dense(output_dim, kernel_initializer='normal', activation='softmax'))					#output layer with softmax as activation function


#print(model.summary())

print ("congrts model defined...")

In [ ]:
# compile the model

model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

# print the model summary

model.summary()

In [ ]:
#Train model

batch_size = 100
epochs = 10

history =model.fit(X_train, Y_train,validation_split=0.2, epochs=epochs, batch_size=batch_size)

print ("parameter tuning done...")

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Accuracy with the epochs

plt.plot(history.history['accuracy'],'r')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training'], loc='center right')
plt.show()

In [ ]:
# Loss with epochs

plt.plot(history.history['loss'],'g')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training'], loc='upper right')
plt.show()